In [54]:
import os
import re
import random

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn import model_selection
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import scipy.linalg
import sklearn.preprocessing
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV

In [2]:
def list_files(dir):
    folders = []
    for root, dirs, files in os.walk(dir):
        for r in dirs:
            folders.append(r)
    return folders

def list_directories(folders):
    images_dirs = []
    for i in range(len(folders)):
        images_d = '/Users/jingyany/Desktop/17Spring/Data558/data competation/train/'+folders[i]+'/'
        images_dirs.append(images_d)
    return images_dirs

def images_lists(images_dirs):
    list_images = []
    for i in range(len(images_dirs)):
        images_dir = images_dirs[i]
        for f in os.listdir(images_dir):
            if re.search('jpg|JPG', f):
                list_images.append(images_dir + f)
    return list_images

In [3]:
model_dir = '/Users/jingyany/models/tutorials/image/imagenet/TUTORIAL_DIR/imagenet'
dir = '/Users/jingyany/Desktop/17Spring/Data558/data competation/train'
folders = list_files(dir)
images_dirs = list_directories(folders)
list_images = images_lists(images_dirs)

In [7]:
def create_graph():
    with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
        
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        if (ind%100 == 0):
        #create_graph()
            print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        features[ind,:] = np.squeeze(predictions)
        labels.append(re.split('_\d+',image.split('/')[-2].split('.')[0])[0])
    labels = list(map(int, labels))

    return features, labels

In [8]:
features,labels = extract_features(list_images)

Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/001.Black_footed_Albatross/Black_Footed_Albatross_0003_796136.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/010.Red_winged_Blackbird/Red_Winged_Blackbird_0025_5342.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/014.Indigo_Bunting/Indigo_Bunting_0056_12637.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/027.Shiny_Cowbird/Shiny_Cowbird_0001_796860.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/031.Black_billed_Cuckoo/Black_Billed_Cuckoo_0042_795308.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/038.Great_Crested_Flycatcher/Great_Crested_Flycatcher_0089_29592.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data competation/train/044.Frigatebird/Frigatebird_0004_43221.jpg...
Processing /Users/jingyany/Desktop/17Spring/Data558/data co

In [11]:
print(features.shape)
labels = np.asarray(labels)
print(labels.shape)

(4320, 2048)
(4320,)


In [12]:
y_train = labels
X_train = features

In [25]:
model_dir = '/Users/jingyany/models/tutorials/image/imagenet/TUTORIAL_DIR/imagenet'
dir_test = '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/'
list_images_test = [dir_test+f for f in os.listdir(dir_test) if re.search('jpg|JPG', f)]
list_images_test

['/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/10.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/100.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1000.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1001.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1002.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1003.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1004.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1005.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1006.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1007.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1008.jpg',
 '/Users/jingyany/Desktop/17Spring/Data558/data competation/test/1009.jpg',
 '/Users/jingyany/

In [30]:
def extract_features_test(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    #labels = []
    test_id = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        if (ind%100 == 0):
            print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        features[ind,:] = np.squeeze(predictions)
        #labels.append(re.split('_\d+',image.split('/')[-2].split('.')[0])[0])
        test_id.append(re.split('_\d+',image.split('/')[-1].split('.')[0])[0])
    #labels = list(map(int, labels))
    test_id = list(map(int, test_id))

    return features, test_id

In [32]:
features_test.shape

(4320, 2048)

In [35]:
X_test = features_test

In [57]:
#Models 1
linearSVC = LinearSVC(C=1.0, loss='squared_hinge', penalty='l2',multi_class='ovr')

In [58]:
linearSVC.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [59]:
y_lnrSVC = linearSVC.predict(X_test)

In [85]:
df_svc = pd.DataFrame({ 'Id' : test_id,
                   'Prediction' : y_lnrSVC})
#df_svc = df_svc.sort_values(['Id'], ascending=[True])                            
df_svc

,Id,Prediction
0,1,50
1,10,147
2,100,72
3,1000,76
4,1001,66
5,1002,199
6,1003,106
7,1004,123
8,1005,48
9,1006,128


In [87]:
df_svc.to_csv('test_labels_lrsvc_new.csv', index=False)